In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn import preprocessing as pp
from matplotlib.font_manager import FontProperties

%matplotlib inline
myfont=FontProperties(fname='/usr/share/fonts/truetype/yahei.ttf',size=14)
mpl.rcParams['axes.unicode_minus']=False
sns.set(font=myfont.get_name())

In [ ]:
1.预测评级变化
2.预测违约风险
问题：


In [4]:
# 数据读取
dateparse0 = lambda dates: pd.datetime.strptime((str(dates)+'0101')[0:8], '%Y%m%d') if not pd.isnull(dates) else None
dateparse1 = lambda dates: pd.datetime.strptime(str(dates)[0:10], '%Y-%m-%d') if not pd.isnull(dates) else None

# 公司基本信息数据
company_df = pd.read_csv('./bond_data/company_info.csv',
                         parse_dates=['成立日期'],
                         infer_datetime_format = True,
                         date_parser=dateparse0)
# 财务数据
bond_df = pd.read_csv('./bond_data/a.csv',
                      parse_dates=['报告期'],
                      infer_datetime_format = True,
                      date_parser=dateparse0)
# 评级数据
level_df = pd.read_csv('./bond_data/level.csv',
                       parse_dates=['评级日期','公告日期'],
                       infer_datetime_format=True,
                       date_parser=dateparse0)
# 债券违约发生数据
bond_fault_df = pd.read_csv('./bond_data/bond_fault.csv',
                         parse_dates=['发生日期'],
                         infer_datetime_format = True,
                         date_parser=dateparse0)
bond_fault_df.rename(columns={'发行人':'公司名称', '发生日期':'违约日期'}, inplace = True)


# 处理舆情信息
# 有错误时间格式记录，需要手工处理
sentiment_df = pd.read_csv('./bond_data/public_sentiment.csv', parse_dates=['pub_date'], date_parser=dateparse1)
# delete none label record
sentiment_df = sentiment_df[sentiment_df['lable_type']!='[]']
# 修改错误拼写列标签, 并拆分标签到多行
sentiment_df = sentiment_df.drop(['lable_type','enterprise_id'], axis=1).join(
    sentiment_df['lable_type'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).rename('label_type'))
sentiment_df['label_type'] = sentiment_df['label_type'].str.replace(pat='[\[\] ]', repl='')
sentiment_df = sentiment_df.drop('label_type', axis=1).join(pd.get_dummies(sentiment_df.loc[~sentiment_df['label_type'].isnull(),'label_type']))
sentiment_df.rename(columns={'enterprise_name':'公司名称'}, inplace = True)

# 标记财务报表类型(年报，半年报，季报)
bond_df['报告类型']=bond_df['报告期'].apply(lambda x: '年报' if x.month==12 else ('半年报' if x.month==6 else '季报'))

# 关联公司基本信息及评级信息
company = pd.merge(company_df, level_df, how='inner', left_on='公司名称', right_on='公司中文名称')

# 删除无用列
company = company.drop(['公司中文简介', '信用评级说明', '法人代表', '总经理', '董事会秘书', '主页', '电子邮箱',
                        '经营范围', '对象ID', '公司ID', '公司中文名称', '主要产品及业务', '办公地址',
                        '评级展望', '债券主体公司id', '城市'], axis=1)

# 设置列类型
company[['公司类别','评级类型','评级机构代码','信用评级','前次信用评级']] = company[['公司类别','评级类型','评级机构代码','信用评级','前次信用评级']].astype(str)
company['员工总数人'] = company[company['员工总数人'].notnull()]['员工总数人'].astype(int)
# 处理评级信息
# 穆迪评级数量很少，所以先匹配norm_level
mudy_level = {"Aaa":100,"Aa1":95,"Aa2":90,"Aa3":85,"A1":80,"A2":75,"A3":70,"Baa1":65,"Baa2":60,\
             "Baa3":55, "Ba1":50,"Ba2":45,"Ba3":40,"B1":35,"B2":30,"B3":25,"Caa1":20,"Caa2":15,\
             "Caa3":10,"Ca":5,"C":0}
norm_level = {"AAA":100,"AA+":95,"AA":90,"AA-":85,"A+":80,"A":75,"A-":70,"BBB+":65,"BBB":60,\
             "BBB-":55, "BB+":50,"BB":45,"BB-":40,"B+":35,"B":30,"B-":25,"CCC":20,"CC":15,\
             "C":10, "RD":5,"D":0}
company['信用评级'] = company['信用评级'].apply(lambda x: norm_level[x] if x in norm_level else mudy_level[x] if x in mudy_level else None)
company['前次信用评级'] = company['前次信用评级'].apply(lambda x: norm_level[x] if x in norm_level else mudy_level[x] if x in mudy_level else None)


In [5]:
# 处理财务报表数据
def process_data(bond):
    # 删除评级日期<报告期的记录+一个月滞后期(暂时没有实现)
    # 按公司名称，报告期分组排序
    bond['sort_id'] =  bond.loc[bond['报告期']<bond['评级日期'],['评级日期']].groupby([bond['公司名称'], bond['报告期']]).rank(ascending=False)

    # 合并评级日期前两期财务指标
    return pd.merge(bond.loc[bond['sort_id'] == 1,bond.columns[0:-1]],
                    bond.loc[bond['sort_id'] == 2,bond.columns[0:-2]],
                    how='left', on=['公司名称','报告期'])


In [9]:
# 根据财务报表类型分别关联公司信息及财务报告
bond_year = pd.merge(bond_df[bond_df['报告类型']=='年报'], company[['公司名称','评级日期']].drop_duplicates(),
                     how='inner', on='公司名称').drop(['公司代码','报告类型'], axis=1)
bond_halfyear = pd.merge(bond_df[bond_df['报告类型']=='半年报'], company[['公司名称','评级日期']],
                         how='inner', on='公司名称').drop(['公司代码','报告类型'], axis=1)
bond_quarter = pd.merge(bond_df[bond_df['报告类型']=='季报'], company[['公司名称','评级日期']].drop_duplicates(),
                        how='inner', on='公司名称').drop(['公司代码','报告类型'], axis=1)

bond_year = process_data(bond_year)
bond_halfyear = process_data(bond_halfyear)
bond_quarter = process_data(bond_quarter)

In [10]:
sentiment_df = pd.merge(company[['评级日期','公司名称']], sentiment_df, how='left', on='公司名称')
# 计算舆情发生日期与评级日期的差值，可以根据这个特征提取一定时间内的舆情信息
sentiment_df['days'] = (sentiment_df['评级日期'] - sentiment_df['pub_date']).dt.days

col = sentiment_df.columns[3:-1].values.tolist()
col += ['公司名称', '评级日期']
# 统计指定时间段舆情信息
b_days = 0
e_days = 31
sentiment_df_30 = sentiment_df.loc[(sentiment_df['days']<e_days) & (sentiment_df['days']>b_days), col]
sentiment_df_30 = sentiment_df_30.groupby(['公司名称','评级日期']).sum().reset_index()

In [65]:
bond_df.columns

Index(['公司代码', '公司名称', '报告期', '基本每股收益', '每股未分配利润', '每股净资产', '净资产收益率', '总资产报酬率',
       '总资产净利率（杜邦分析）', '经营活动净收益/利润总额', '营业外收支净额/利润总额', '销售商品提供劳务收到的现金/营业收入',
       '经营活动产生的现金流量净额/营业收入', '资产负债率', '权益乘数(用于杜邦分析) ', '流动资产/总资产',
       '归属于母公司的股东权益/全部投入资本', '流动比率', '速动比率', '产权比率', '经营活动产生的现金流量净额/负债合计',
       '存货周转率', '应收账款周转率', '总资产周转率', '同比增长率-基本每股收益(%)', '同比增长率-营业利润(%)',
       '同比增长率-经营活动产生的现金流量净额(%)', '报告类型'],
      dtype='object')

In [12]:
# 统计有违约的公司与有评级的公司的关联信息（只有8个违约企业可以关联上）
print(level_df['公司中文名称'].nunique())
print(pd.merge(bond_fault_df, level_df, how='left', left_on='公司名称', right_on='公司中文名称')[['公司名称','公司中文名称']].drop_duplicates().sort_values(by=['公司中文名称']).values)

# 统计有违约的公司与有基本信息的公司的关联信息（都可以关联上）
print(company_df['公司名称'].nunique())
print(pd.merge(bond_fault_df, company_df, how='left', on='公司名称')[['公司名称','公司ID']].drop_duplicates().sort_values(by=['公司ID']).values)

# 统计有违约的公司与有财务信息的公司的关联信息（只有19个违约企业可以关联上）
print(bond_df['公司名称'].nunique())
print(pd.merge(bond_fault_df, bond_df, how='left', on='公司名称')[['公司名称','公司代码']].drop_duplicates().sort_values(by=['公司代码']).values)

# 统计有违约的公司与有舆情信息的公司的关联关系（只有37个违约企业可以关联上）
fault_sentiment_df = pd.read_csv('./bond_data/public_sentiment_fault.csv', parse_dates=['发布日期'], date_parser=dateparse1)
print(fault_sentiment_df['企业名称'].nunique())
print(pd.merge(bond_fault_df, fault_sentiment_df, how='left', left_on='公司名称', right_on='企业名称')[['公司名称','企业名称']].drop_duplicates().sort_values(by=['企业名称']).values)



3895
[['中煤集团山西华昱能源有限公司' '中煤集团山西华昱能源有限公司']
 ['五洋建设集团股份有限公司' '五洋建设集团股份有限公司']
 ['亿利资源集团有限公司' '亿利资源集团有限公司']
 ['信阳市弘昌管道燃气工程有限责任公司' '信阳市弘昌管道燃气工程有限责任公司']
 ['内蒙古博源控股集团有限公司' '内蒙古博源控股集团有限公司']
 ['华盛江泉集团有限公司' '华盛江泉集团有限公司']
 ['四川省煤炭产业集团有限责任公司' '四川省煤炭产业集团有限责任公司']
 ['大连机床集团有限责任公司' '大连机床集团有限责任公司']
 ['中国城市建设控股集团有限公司' nan]
 ['甘肃宏良皮业股份有限公司' nan]
 ['上海市建设机电安装有限公司' nan]
 ['丹东港集团有限公司' nan]
 ['湖州厉华妤婕联合纺织有限公司' nan]
 ['东北特殊钢集团有限责任公司' nan]
 ['山东山水水泥集团有限公司' nan]
 ['内蒙古奈伦集团股份有限公司' nan]
 ['春和集团有限公司' nan]
 ['珠海中富实业股份有限公司' nan]
 ['广州华工百川科技有限公司' nan]
 ['惠州侨兴电信工业有限公司' nan]
 ['惠州侨兴电讯工业有限公司' nan]
 ['河北省物流产业集团有限公司' nan]
 ['山东迪浩耐磨管道股份有限公司' nan]
 ['莱芜信通印刷设备有限公司' nan]
 ['武汉国裕物流产业集团有限公司' nan]
 ['百花医药集团股份有限公司' nan]
 ['亚邦投资控股集团有限公司' nan]
 ['金乡县华光食品进出口有限公司' nan]
 ['华珠(泉州)鞋业有限公司' nan]
 ['上海云峰(集团)有限公司' nan]
 ['河南佳源乳业股份有限公司' nan]
 ['东兴金满堂商贸有限公司' nan]
 ['甘肃华协农业生物科技股份有限公司' nan]
 ['鄂尔多斯市益通路桥有限公司' nan]
 ['保定天威英利新能源有限公司' nan]
 ['天津市泰亨气体有限公司' nan]
 ['广西有色金属集团有限公司' nan]
 ['保定天威集团有限公司' nan]
 ['南京雨润食品有限公司' nan]
 ['江苏中联物流股份有限公司' nan]
 ['中成新

In [27]:
# 目前有的数据
# 财务数据 bond_year(按财务周期分类)
# 公司信息 company
# 违约数据 bond_fault_df
# 與情数据 sentiment_df_30(按距离评级日期时间长短进行分类)
print(company.shape)
# 关联违约发生数据（有违约日期的数据即为有债券违约的公司）
result = pd.merge(company, bond_fault_df.drop_duplicates(), how='left', on='公司名称')
print(result.shape)
# 关联舆情发生在评级日期前的舆情数据
result = pd.merge(result, sentiment_df_30, how='left', on=['公司名称','评级日期'])
print(result.shape)
# 标记发生违约的记录
# 规则：有违约日期且违约日期>评级日期并且违约日期最接近评级日期
# bond['sort_id'] =  bond.loc[bond['报告期']<bond['评级日期'],['评级日期']].groupby([bond['公司名称'], bond['报告期']]).rank(ascending=False)
result['sort_id'] = result.loc[result['评级日期']<result['违约日期'],['违约日期']].groupby([result['公司名称'],result['评级日期']]).rank(ascending=True)
# 关联财务数据
result = pd.merge(result[(result['sort_id']==1) | (result['sort_id']!=None)], bond_year, how='left', on=['公司名称','评级日期'])
print(result.shape)


(5174, 16)
(5192, 17)
(5192, 193)
(9055, 243)


In [34]:
pd.merge(company, bond_fault_df.drop_duplicates(), how='inner', on='公司名称').shape

(31, 17)

In [33]:
result.loc[result['sort_id'].notnull(), ['sort_id']].shape

(10, 1)

In [14]:
print('company_year:', company_year.shape)
print('sentiment_df:', sentiment_df_30.shape)
print('company:', company.shape)
print('company_halfyear:', company_halfyear.shape)

company_year: (9639, 220)
sentiment_df: (167, 178)
company: (5174, 16)
company_halfyear: (12787, 43)


In [19]:
a=sentiment_df.groupby([sentiment_df['公司名称'],sentiment_df['评级日期']]).sum().reset_index()
a.columns


Index(['公司名称', '评级日期', '', ' 风险->个人风险->高管动向', ' 风险->个人风险->高管风险',
       ' 风险->企业风险->事故->泄漏事故', ' 风险->企业风险->事故->火灾事故', ' 风险->企业风险->事故->爆炸事故',
       ' 风险->企业风险->产品风险->不合格产品', ' 风险->企业风险->产品风险->产品差',
       ...
       '风险->行业风险->行业分析->趋势变化', '风险->行业风险->行业分析->黑天鹅', '风险->行业风险->行业调整->产能过剩',
       '风险->行业风险->行业调整->产能问题', '风险->行业风险->行业调整->市场面临考验',
       '风险->行业风险->行业调整->空间有限', '风险->行业风险->行业调整->行业改革', '风险->行业风险->行业调整->行业调整',
       '风险->行业风险->行业调整->需求萎缩', 'days'],
      dtype='object', length=307)

In [ ]:
思路：
增加发行时主体评级，以及当前信用评级，将信用评级数值化，并计算信用评级差
可以关联同一家公司的多个违约记录，以便增加样本数量
关联舆情分类指标（一个月，一个季度，半年，一年）统计各时间段内舆情分类发生次数